In [25]:
import requests
from bs4 import BeautifulSoup
import MySQLdb
import pandas as pd
from selenium import webdriver

In [138]:
url = 'https://www.forbes.com/global2000/list/#country:United States'

In [147]:
driver = webdriver.Chrome(executable_path='/Users/matrixxt/Downloads/apps/chromedriver')
# 换用无头 webdriver
#driver = webdriver.PhantomJS('/Users/matrixxt/Downloads/apps/phantomjs-2.1.1-macosx/bin/phantomjs')

In [148]:
driver.get(url)

In [158]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
driver.close()
driver.quit()

In [159]:
items = soup.find_all('tr', class_='data')

In [279]:
list_info = []
for item in items[:-1]:
    rank = item.find(class_='rank').text
    name = item.find(class_='name').text
    suburl = item.find(class_='name').a.get('href')
    country , revenue, profits, assets, market_value =\
    [td.text for td in item.find_all('td', class_= False)]
    dict_info = {'Name': name,
                'Rank': int(rank[1:]),
                'Suburl': suburl,
                'Country': country,
                'Revenue': revenue,
                 'Profits': profits,
                 'Assets': assets,
                 'MarketValue': market_value}
    list_info.append(dict_info)

In [335]:
for dct in list_info:
    suburl = 'https://www.forbes.com' + dct['Suburl']
    try:
        req = requests.get(suburl)
        soup = BeautifulSoup(req.text, 'lxml')
        data = soup.find(class_='data has_image')
        dd_list = data.find_all('dd')
        dct['Industry'] = dd_list[1].text
        dct['Founded'] = dd_list[2].text
        dct['CEO'] = dd_list[4].text
        dct['Employees'] = dd_list[6].text
        dct['Headquarters'] = dd_list[8].text
    except:
        pass

In [ ]:
df = pd.DataFrame(data=list_info)
df.set_index('Name',inplace=True)

In [ ]:
df.to_csv('USATop200Companies.csv')

In [354]:
df.sort_values('Rank')

Assets                          CEO  \
Name                                                                       
Berkshire Hathaway                 $620.9 B               Warren Buffett   
JPMorgan Chase                     $2,513 B                  Jamie Dimon   
Wells Fargo                      $1,943.4 B                Timothy Sloan   
Bank of America                  $2,196.8 B               Brian Moynihan   
Apple                              $331.1 B                     Tim Cook   
AT&T                               $403.8 B           Randall Stephenson   
Citigroup                        $1,795.1 B               Michael Corbat   
ExxonMobil                         $330.3 B                 Darren Woods   
General Electric                   $365.2 B                John Flannery   
Wal-Mart Stores                    $198.8 B                  C. McMillon   
Verizon Communications             $244.2 B                Lowell McAdam   
Microsoft                          $224.6 B                Satya Nadella   
Alphabet                           $167.5 B                Lawrence Page   
Comcast                            $180.5 B                Brian Roberts   
Johnson & Johnson                  $141.2 B                  Alex Gorsky   
UnitedHealth Group                   $125 B               David Wichmann   
General Motors                     $221.7 B                   Mary Barra   
IBM                                $117.5 B             Virginia Rometty   
Procter & Gamble                     $117 B                 David Taylor   
Pfizer                             $171.6 B                     Ian Read   
Goldman Sachs Group                $860.2 B              Lloyd Blankfein   
Intel                              $113.3 B               Brian Krzanich   
Cisco Systems                      $126.2 B                Chuck Robbins   
Morgan Stanley                       $816 B                 James Gorman   
Ford Motor                           $238 B                James Hackett   
CVS Health                          $94.5 B                  Larry Merlo   
Walt Disney                         $91.6 B                  Robert Iger   
Prudential Financial                 $784 B  John Robert Strangfeld, Jr.   
Oracle                             $125.4 B     Safra A. Catz, Mark Hurd   
Boeing                                $90 B                 Leanne Caret   
...                                     ...                          ...   
Ecolab                              $18.3 B            Douglas Baker Jr.   
Omnicom Group                       $23.2 B                    John Wren   
Loews                               $76.6 B                      Jim MBA   
ConocoPhillips                      $89.8 B                   Ryan Lance   
Baxter International                $15.5 B                  Joe Almeida   
Praxair                             $19.3 B                Stephen Angel   
Simon Property Group                $31.1 B                  David Simon   
Cummins                               $15 B               Tom Linebarger   
Viacom                              $23.3 B                Robert Bakish   
Public Service Enterprise Group     $40.1 B                   Ralph Izzo   
Regions Financial                  $126.7 B               O. B. Hall Jr.   
CenturyLink                           $47 B                    Glen Post   
Citizens Financial Group           $150.6 B               Bruce van Saun   
Ally Financial                     $163.7 B                Jeffrey Brown   
DTE Energy                            $32 B              Gerard Anderson   
Unum                                $63.6 B                Rick McKenney   
Best Buy                            $14.2 B                  Hubert Joly   
Northern Trust                     $123.9 B              Michael O'Grady   
CME Group                           $69.4 B               Terrence Duffy   
American Tower                      $30.9 B            James Taiclet Jr.   
Newell Br

参考：
- [美国２０１７财年财政赤字创２０１３年来新高-新华网](http://www.xinhuanet.com/world/2017-10/21/c_1121836189.htm)
- [The World’s Biggest Public Companies List](https://www.forbes.com/global2000/list/#country:United%20States)
- [盘点selenium phantomJS使用的坑 - CSDN博客](https://blog.csdn.net/FontThrone/article/details/71390739)
- [python爬虫积累（一）--------selenium+python+PhantomJS的使用 - 今孝 - 博客园](http://www.cnblogs.com/jinxiao-pu/p/6677782.html)

In [357]:
revenue = df['Revenue'].map(lambda x: float(x[1:-1]))

In [360]:
revenue.mean()

43.767999999999994